In [ ]:
import numpy as np
import json
import openai
import seaborn as sn

load_dotenv()
load_dotenv("variables.env")
openai.api_key = os.getenv("ANTHROPIC_API_KEY")

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(model=model, messages=messages,temperature=0)

    return response.choices[0].message["content"]


prompt1 = 'You are presented with a pair of words consisting of cueword and guessword.'+\
' Given the pair of words, please predict binary label whether the guessword is come up with a human or an AI.'+\
' The pair of words is represented as <cueword, guessword>. For example, <apple, phone> '+\
'where apple is cueword and phone is guessword.'+\
' Please output whether the agent coming up with the guessword is a human or AI in the following example: '


cue_words_list = np.load('Data_all/all_cue_words_october_1.npy')
CueGuessWords = {}

for w in cue_words_list:
    #print(w)
    CueGuessWords[w] = {'human':[],'AI':[]}

    
numfiles = 150
numfolders = 5

for identity in ['human','AI']:
    for fo in range(numfolders):
        for fi in range(numfiles):
            filename = identity + '/' + str(fo) + '/' + str(fi) + '.html'

            with open('./dataset/' + filename) as f:
                lines = f.readlines()
                #print(lines)
                cueword = lines[0].split('<p>Cue: ')[1].split(';')[0]
                #print(cueword)
                guessword = lines[0].split('; Association: ')[1].split(' </p>')[0]
                #print(guessword)
                CueGuessWords[cueword][identity].append(guessword)

#print(CueGuessWords)

for w in cue_words_list:
    #print(w)
    strlistH = CueGuessWords[w]['human']
    strlistM = CueGuessWords[w]['AI']
    flag = bool(set(strlistH).intersection(strlistM))
#     if not flag:
#         print(w)
#         print(strlistH)
#         print(strlistM)
    CueGuessWords[w]['flag'] = flag

    
H_responses_all = [] # all cases together
AI_responses_all = []
    
for w in cue_words_list:
    print('There are ' + str(len(cue_words_list)) + ' cue words.')
    if not CueGuessWords[w]['flag']:
        cueword = w
        
        for identity in ['human','AI']:
            for guessword in CueGuessWords[cueword][identity]:
                prompt2 = '<'+cueword+','+guessword+'>'
                entireprompt = prompt1 + prompt2
                print(entireprompt)
                
                response = get_completion(entirePrompt)
                print(response)
                
                
                #print(myList)
                if 'AI' in response:
                    conv1_Aclass = 'AI'
                elif 'Human' in response or 'human' in response:
                    conv1_Aclass = 'human'
                else:
                    print("Not found!")
                    continue
                    
                if identity == 'human':
                    
                    if conv1_Aclass == 'human':
                        H_responses_all.append(1)
                    else:
                        H_responses_all.append(0)
                        
                else:
                    if conv1_Aclass == 'human':
                        AI_responses_all.append(0)
                    else:
                        AI_responses_all.append(1)
        

In [ ]:
print(H_responses_all) # all cases together
print(AI_responses_all)

modelist = ['Human (all)',           
           'Machine (all)']

###################3compute confusion matrix
result_conf_mat = np.zeros((len(modelist),2),'float32')
result_conf_mat[0][0] = H_responses_all
result_conf_mat[0][1] = 1-H_responses_all
result_conf_mat[1][0] = 1-AI_responses_all
result_conf_mat[1][1] = AI_responses_all
print(result_conf_mat)
    
df_cm = pd.DataFrame(result_conf_mat, index = [i for i in modelist],
              columns = [i for i in ['Human','Machine']])
plt.figure(figsize = (1.2,2.5))
sn.heatmap(df_cm, annot=True,robust=True, cmap='RdBu_r', vmin=0, vmax=1)

plt.xlabel("Predicted")
plt.ylabel("Ground Truth")

key = 'fewshotAIjudge'
folderpath = './plots/Task2_' + key + '_' #for saving the plots
plotname = 'fewshotAI_judge'
plt.savefig(folderpath + plotname + '_overall_confmat.eps', format='eps', bbox_inches='tight')
plt.savefig(folderpath + plotname + '_overall_confmat.png', bbox_inches='tight')
#     plt.savefig('../' + plotname + '_confmat.png', bbox_inches='tight')
#     plt.savefig('Human_judge_confmatGiorgia'+condition_chosen+'.pdf', bbox_inches='tight')
#     plt.savefig('Human_judge_confmatGiorgia'+condition_chosen+'.png', bbox_inches='tight')
plt.show()